In [ ]:
CREATE TABLE users (
    id SERIAL PRIMARY KEY, 
    Username TEXT NOT NULL UNIQUE 
);


In [ ]:
CREATE TABLE projects (
    id SERIAL PRIMARY KEY,
    Name TEXT NOT NULL,
    user_id INTEGER REFERENCES users(id)
);

In [ ]:
CREATE TABLE experiments (
    id SERIAL PRIMARY KEY,
    Name TEXT NOT NULL,
    project_id INTEGER REFERENCES projects(id)
);

In [ ]:
CREATE TABLE documents (
    id SERIAL PRIMARY KEY,
    Name TEXT NOT NULL,
    experiment_id INTEGER REFERENCES experiments(id)
);

In [ ]:
CREATE TABLE probes (
    id SERIAL PRIMARY KEY,
    description TEXT NOT NULL,
    document_id INTEGER REFERENCES documents(id)
);

In [ ]:
CREATE TABLE samples (
    id SERIAL PRIMARY KEY,
    probe_id INTEGER REFERENCES probes(id)
    sample_count INTEGER,
    last_sample_time TIMESTAMPTZ,
    current_sampling_interval INTEGER,
    min_sampling_interval INTEGER,
    max_sampling_interval INTEGER
);

In [ ]:
CREATE TABLE spectra (
    id SERIAL PRIMARY KEY,
    sample_id INTEGER REFERENCES samples(id), 
    type TEXT CHECK (type IN ('raw', 'background', 'treated')),
    file_path TEXT NOT NULL,
    recorded_at TIMESTAMPTZ DEFAULT NOW()
);